## AlexNet on CIFAR 10

In [1]:
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
import tqdm
import torchinfo


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda', index=0)

In [3]:
'''
Step 1:
'''

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

train_dataset = datasets.CIFAR10(root='./cifar_10data/',
                                 train=True, 
                                 transform=transform,
                                 download=True)

test_dataset = datasets.CIFAR10(root='./cifar_10data/',
                                train=False, 
                                transform=transforms.ToTensor())
    

Files already downloaded and verified


In [4]:
'''
Step 2
'''

class AlexNet(nn.Module) :
    
    def __init__(self, num_class=10) :
        super(AlexNet, self).__init__()
        
        self.conv_layer1 = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=4),
                nn.ReLU(),
                nn.Conv2d(96, 96, kernel_size=3),
                nn.ReLU()
                )
        self.conv_layer2 = nn.Sequential(
                nn.Conv2d(96, 256, kernel_size=5, padding=2),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2)
                )
        self.conv_layer3 = nn.Sequential(
                nn.Conv2d(256, 384, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(384, 384, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(384, 256, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2)
                )
        
        self.fc_layer1 = nn.Sequential(
                nn.Dropout(),
                nn.Linear(9216, 4096),
                nn.ReLU(),
                nn.Dropout(),  #p=0.5 by default
                nn.Linear(4096, 4096),
                nn.ReLU(),     #p=0.5 by default
                nn.Linear(4096, 10)
                )
    
    def forward(self, x) :
        output = self.conv_layer1(x)
        output = self.conv_layer2(output)
        output = self.conv_layer3(output)
        output = output.view(-1, 9216)
        output = self.fc_layer1(output)
        return output

    

In [5]:
'''
Step 3
'''
model = AlexNet().to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, weight_decay=0.00005)

In [6]:
torchinfo.summary(model,(200,3,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [200, 10]                 --
├─Sequential: 1-1                        [200, 96, 27, 27]         --
│    └─Conv2d: 2-1                       [200, 96, 29, 29]         4,704
│    └─ReLU: 2-2                         [200, 96, 29, 29]         --
│    └─Conv2d: 2-3                       [200, 96, 27, 27]         83,040
│    └─ReLU: 2-4                         [200, 96, 27, 27]         --
├─Sequential: 1-2                        [200, 256, 13, 13]        --
│    └─Conv2d: 2-5                       [200, 256, 27, 27]        614,656
│    └─ReLU: 2-6                         [200, 256, 27, 27]        --
│    └─MaxPool2d: 2-7                    [200, 256, 13, 13]        --
├─Sequential: 1-3                        [200, 256, 6, 6]          --
│    └─Conv2d: 2-8                       [200, 384, 13, 13]        885,120
│    └─ReLU: 2-9                         [200, 384, 13, 13]        -

In [7]:
torch.cuda.empty_cache()

In [8]:
'''
Step 4
'''
model.train()
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)

import time
start = time.time()
for epoch in tqdm.tqdm(range(100)):
    print("{}th epoch starting.".format(epoch))
    for i, (images, labels) in enumerate(train_loader) :
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        train_loss = loss_function(model(images), labels)
        train_loss.backward()

        optimizer.step()

    print ("Epoch [{}] Loss: {:.4f}".format(epoch+1, train_loss.item()))

end = time.time()
print("Time ellapsed in training is: {}".format(end - start))

  0%|          | 0/100 [00:00<?, ?it/s]

0th epoch starting.


  1%|          | 1/100 [00:14<24:01, 14.56s/it]

Epoch [1] Loss: 2.3025
1th epoch starting.


  2%|▏         | 2/100 [00:28<23:21, 14.30s/it]

...

Epoch [98] Loss: 0.1547
98th epoch starting.


 99%|█████████▉| 99/100 [23:21<00:14, 14.13s/it]

Epoch [99] Loss: 0.0925
99th epoch starting.


100%|██████████| 100/100 [23:35<00:00, 14.15s/it]

Epoch [100] Loss: 0.0703
Time ellapsed in training is: 1415.2418777942657


In [9]:
'''
Step 5
'''
model.eval()
test_loss, correct, total = 0, 0, 0

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
with torch.no_grad():  #using context manager
    for images, labels in test_loader :
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        test_loss += loss_function(output, labels).item()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()

        total += labels.size(0)

print('[Test set] Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss /total, correct, total,
        100. * correct / total))

[Test set] Average loss: 0.0039, Accuracy: 8953/10000 (89.53%)

